<a href="https://colab.research.google.com/github/ashkanb77/Activity-Recognition/blob/main/Activity_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget http://archive.ics.uci.edu/ml/machine-learning-databases/00506/casas-dataset.zip

--2022-02-14 17:57:05--  http://archive.ics.uci.edu/ml/machine-learning-databases/00506/casas-dataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13952153033 (13G) [application/x-httpd-php]
Saving to: ‘casas-dataset.zip’

casas-dataset.zip   100%[===================>]  12.99G  44.0MB/s    in 5m 2s   

2022-02-14 18:02:07 (44.0 MB/s) - ‘casas-dataset.zip’ saved [13952153033/13952153033]



In [ ]:
import os
import shutil
from tqdm import tqdm

In [ ]:
zip_address = 'casas-dataset.zip'
extracted_address = 'casas-dataset-extracted'
unzip_command = '7z e {0} -o{1} {2} -x!{2}/{2}.rawdata.*'
rm_command = 'rm -r {0}'

try:
    os.mkdir(extracted_address)
except:
    shutil.rmtree(extracted_address)
    os.mkdir(extracted_address)

os.system(f'7z e {zip_address} -o{extracted_address} README.txt')
for i in tqdm(range(101, 131)):
    os.system(unzip_command.format(zip_address, f'{extracted_address}/csh{i}', f'csh{i}'))
    os.rmdir(f'{extracted_address}/csh{i}/csh{i}')

100%|██████████| 30/30 [00:48<00:00,  1.60s/it]


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import deque
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest


In [2]:
dataset = pd.read_csv('drive/MyDrive/datasets/casas-dataset-extracted/csh111/csh111.ann.features.csv')

In [3]:
dataset.head()

,lastSensorEventHours,lastSensorEventSeconds,lastSensorDayOfWeek,windowDuration,timeSinceLastSensorEvent,prevDominantSensor1,prevDominantSensor2,lastSensorID,lastSensorLocation,lastMotionLocation,complexity,activityChange,areaTransitions,numDistinctSensors,sensorCount-Bathroom,sensorCount-Bedroom,sensorCount-Chair,sensorCount-DiningRoom,sensorCount-Hall,sensorCount-Ignore,sensorCount-Kitchen,sensorCount-LivingRoom,sensorCount-Office,sensorCount-OutsideDoor,sensorCount-WorkArea,sensorElTime-Bathroom,sensorElTime-Bedroom,sensorElTime-Chair,sensorElTime-DiningRoom,sensorElTime-Hall,sensorElTime-Ignore,sensorElTime-Kitchen,sensorElTime-LivingRoom,sensorElTime-Office,sensorElTime-OutsideDoor,sensorElTime-WorkArea,activity
0,0.0,1258.0,2.0,698.0,1.0,0.0,0.0,1.0,1.0,1.0,0.650022,0.028653,0.0,0.0,0.0,28.62,0.0,0.0,0.0,5.73,0.0,0.0,0.0,0.0,0.0,86400.0,0.0,86400.0,86400.0,86400.0,1.571419,86400.0,86400.0,86400.0,86400.0,86400.0,Sleep
1,0.0,1260.0,2.0,699.0,2.0,0.0,0.0,1.0,1.0,1.0,0.650022,0.028612,0.0,0.0,0.0,28.57,0.0,0.0,0.0,5.78,0.0,0.0,0.0,0.0,0.0,86400.0,0.0,86400.0,86400.0,86400.0,2.852767,86400.0,86400.0,86400.0,86400.0,86400.0,Sleep
2,0.0,1282.0,2.0,720.0,22.0,0.0,0.0,1.0,1.0,1.0,0.650022,0.029167,0.0,0.0,0.0,28.52,0.0,0.0,0.0,5.83,0.0,0.0,0.0,0.0,0.0,86400.0,0.0,86400.0,86400.0,86400.0,25.571251,86400.0,86400.0,86400.0,86400.0,86400.0,Sleep
3,0.0,1283.0,2.0,719.0,1.0,0.0,0.0,1.0,1.0,1.0,0.650022,0.027816,0.0,0.0,0.0,28.47,0.0,0.0,0.0,5.88,0.0,0.0,0.0,0.0,0.0,86400.0,0.0,86400.0,86400.0,86400.0,26.665044,86400.0,86400.0,86400.0,86400.0,86400.0,Sleep
4,0.0,1329.0,2.0,756.0,46.0,0.0,0.0,1.0,1.0,1.0,0.650022,0.017196,0.0,0.0,0.0,28.42,0.0,0.0,0.0,5.93,0.0,0.0,0.0,0.0,0.0,86400.0,0.0,86400.0,86400.0,86400.0,72.605688,86400.0,86400.0,86400.0,86400.0,86400.0,Sleep


In [4]:
len(dataset['activity'].unique())

35

In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351324 entries, 0 to 351323
Data columns (total 37 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   lastSensorEventHours      351324 non-null  float64
 1   lastSensorEventSeconds    351324 non-null  float64
 2   lastSensorDayOfWeek       351324 non-null  float64
 3   windowDuration            351324 non-null  float64
 4   timeSinceLastSensorEvent  351324 non-null  float64
 5   prevDominantSensor1       351324 non-null  float64
 6   prevDominantSensor2       351324 non-null  float64
 7   lastSensorID              351324 non-null  float64
 8   lastSensorLocation        351324 non-null  float64
 9   lastMotionLocation        351324 non-null  float64
 10  complexity                351324 non-null  float64
 11  activityChange            351324 non-null  float64
 12  areaTransitions           351324 non-null  float64
 13  numDistinctSensors        351324 non-null  f

In [6]:
dataset.isna().sum()

lastSensorEventHours        0
lastSensorEventSeconds      0
lastSensorDayOfWeek         0
windowDuration              0
timeSinceLastSensorEvent    0
prevDominantSensor1         0
prevDominantSensor2         0
lastSensorID                0
lastSensorLocation          0
lastMotionLocation          0
complexity                  0
activityChange              0
areaTransitions             0
numDistinctSensors          0
sensorCount-Bathroom        0
sensorCount-Bedroom         0
sensorCount-Chair           0
sensorCount-DiningRoom      0
sensorCount-Hall            0
sensorCount-Ignore          0
sensorCount-Kitchen         0
sensorCount-LivingRoom      0
sensorCount-Office          0
sensorCount-OutsideDoor     0
sensorCount-WorkArea        0
sensorElTime-Bathroom       0
sensorElTime-Bedroom        0
sensorElTime-Chair          0
sensorElTime-DiningRoom     0
sensorElTime-Hall           0
sensorElTime-Ignore         0
sensorElTime-Kitchen        0
sensorElTime-LivingRoom     0
sensorElTi

In [7]:
dataset.isnull().sum()

lastSensorEventHours        0
lastSensorEventSeconds      0
lastSensorDayOfWeek         0
windowDuration              0
timeSinceLastSensorEvent    0
prevDominantSensor1         0
prevDominantSensor2         0
lastSensorID                0
lastSensorLocation          0
lastMotionLocation          0
complexity                  0
activityChange              0
areaTransitions             0
numDistinctSensors          0
sensorCount-Bathroom        0
sensorCount-Bedroom         0
sensorCount-Chair           0
sensorCount-DiningRoom      0
sensorCount-Hall            0
sensorCount-Ignore          0
sensorCount-Kitchen         0
sensorCount-LivingRoom      0
sensorCount-Office          0
sensorCount-OutsideDoor     0
sensorCount-WorkArea        0
sensorElTime-Bathroom       0
sensorElTime-Bedroom        0
sensorElTime-Chair          0
sensorElTime-DiningRoom     0
sensorElTime-Hall           0
sensorElTime-Ignore         0
sensorElTime-Kitchen        0
sensorElTime-LivingRoom     0
sensorElTi

In [8]:
skb = SelectKBest(k=10)
dataset_values = skb.fit_transform(dataset.drop(['activity'], axis=1).values, dataset['activity'])

/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [13 16 17 18 22 27 28 29 33] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [9]:
scaler = StandardScaler()
dataset_values = scaler.fit_transform(dataset_values)

In [10]:
yy = dataset['activity'].values

In [11]:
d = deque(maxlen=15)
X = []
y = []
for i in range(len(dataset_values)):
  row = dataset_values[i]
  d.append([j for j in row])  
  if len(d) == 15:
    X.append([j for j in d])
    y.append(yy[i])



In [12]:
X = np.array(X)
y = np.array(y)

In [13]:
X.shape

(351310, 15, 10)

In [14]:
np.unique(y, return_counts=True)

(array(['Bathe', 'Bed_Toilet_Transition', 'Cook', 'Cook_Breakfast',
        'Cook_Dinner', 'Cook_Lunch', 'Dress', 'Drink', 'Eat',
        'Eat_Breakfast', 'Eat_Dinner', 'Eat_Lunch', 'Enter_Home',
        'Entertain_Guests', 'Evening_Meds', 'Groom', 'Leave_Home',
        'Morning_Meds', 'Other_Activity', 'Personal_Hygiene', 'Phone',
        'Read', 'Relax', 'Sleep', 'Step_Out', 'Toilet',
        'Wash_Breakfast_Dishes', 'Wash_Dinner_Dishes', 'Wash_Dishes',
        'Wash_Lunch_Dishes', 'Watch_TV', 'Work', 'Work_At_Desk',
        'Work_At_Table', 'Work_On_Computer'], dtype='<U21'),
 array([  1268,    987,    544,   2480,   4190,   1894,  11783,   3664,
           372,   3100,    694,   1613,   1722,  75414,    920,   3289,
          2072,    939, 112806,  10163,    521,  13583,   1208,  16542,
           632,   6489,    894,   1022,   1597,    511,   9625,    926,
           849,   1733,  55264]))

In [15]:
y = pd.get_dummies(y).values

In [16]:
y.shape

(351310, 35)

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

In [ ]:
model = Sequential([
                    LSTM(500,  input_shape=(X_train.shape[1:])),
                    # LSTM(300),
                    Dense(2000, 'tanh'),
                    Dense(1000, 'tanh'),
                    Dense(35, 'softmax', )
])

In [ ]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_7 (LSTM)               (None, 500)               1022000   
                                                                 
 dense_21 (Dense)            (None, 2000)              1002000   
                                                                 
 dense_22 (Dense)            (None, 1000)              2001000   
                                                                 
 dense_23 (Dense)            (None, 35)                35035     
                                                                 
Total params: 4,060,035
Trainable params: 4,060,035
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
mchp = ModelCheckpoint('drive/MyDrive/datasets/casas-dataset-extracted/csh117/model.h5', save_best_only=True)

In [ ]:
history = model.fit(X_train, y_train, epochs=20, batch_size=500, validation_data=(X_test, y_test), callbacks=[mchp])

Epoch 1/20
563/563 [==============================] - 35s 58ms/step - loss: 1.2032 - accuracy: 0.6097 - val_loss: 1.0871 - val_accuracy: 0.6378
Epoch 2/20
563/563 [==============================] - 32s 57ms/step - loss: 1.0436 - accuracy: 0.6537 - val_loss: 1.0232 - val_accuracy: 0.6629
Epoch 3/20
563/563 [==============================] - 32s 56ms/step - loss: 0.9859 - accuracy: 0.6696 - val_loss: 0.9753 - val_accuracy: 0.6699
Epoch 4/20
563/563 [==============================] - 32s 56ms/step - loss: 0.9419 - accuracy: 0.6807 - val_loss: 0.9454 - val_accuracy: 0.6811
Epoch 5/20
563/563 [==============================] - 32s 56ms/step - loss: 0.8964 - accuracy: 0.6936 - val_loss: 0.8817 - val_accuracy: 0.6986
Epoch 6/20
563/563 [==============================] - 32s 56ms/step - loss: 0.8477 - accuracy: 0.7079 - val_loss: 0.8327 - val_accuracy: 0.7140
Epoch 7/20
563/563 [==============================] - 32s 57ms/step - loss: 0.7981 - accuracy: 0.7234 - val_loss: 0.7929 - val_accuracy:

In [ ]:
history = model.fit(X_train, y_train, epochs=20, batch_size=500, validation_data=(X_test, y_test), callbacks=[mchp])

Epoch 1/20
563/563 [==============================] - 32s 56ms/step - loss: 0.4358 - accuracy: 0.8459 - val_loss: 0.5426 - val_accuracy: 0.8195
Epoch 2/20
563/563 [==============================] - 31s 55ms/step - loss: 0.4325 - accuracy: 0.8475 - val_loss: 0.5442 - val_accuracy: 0.8173
Epoch 3/20
563/563 [==============================] - 32s 56ms/step - loss: 0.4193 - accuracy: 0.8518 - val_loss: 0.5410 - val_accuracy: 0.8176
Epoch 4/20
563/563 [==============================] - 32s 56ms/step - loss: 0.4147 - accuracy: 0.8538 - val_loss: 0.5339 - val_accuracy: 0.8226
Epoch 5/20
563/563 [==============================] - 32s 56ms/step - loss: 0.4083 - accuracy: 0.8564 - val_loss: 0.5322 - val_accuracy: 0.8245
Epoch 6/20
563/563 [==============================] - 32s 56ms/step - loss: 0.3968 - accuracy: 0.8601 - val_loss: 0.5242 - val_accuracy: 0.8272
Epoch 7/20
563/563 [==============================] - 31s 55ms/step - loss: 0.3924 - accuracy: 0.8616 - val_loss: 0.5248 - val_accuracy:

In [ ]:
history = model.fit(X_train, y_train, epochs=20, batch_size=500, validation_data=(X_test, y_test), callbacks=[mchp])

Epoch 1/20
563/563 [==============================] - 31s 55ms/step - loss: 0.3130 - accuracy: 0.8894 - val_loss: 0.5271 - val_accuracy: 0.8374
Epoch 2/20
563/563 [==============================] - 31s 55ms/step - loss: 0.3128 - accuracy: 0.8896 - val_loss: 0.5133 - val_accuracy: 0.8434
Epoch 3/20
563/563 [==============================] - 32s 56ms/step - loss: 0.3076 - accuracy: 0.8918 - val_loss: 0.5050 - val_accuracy: 0.8453
Epoch 4/20
563/563 [==============================] - 31s 55ms/step - loss: 0.2998 - accuracy: 0.8940 - val_loss: 0.5086 - val_accuracy: 0.8456
Epoch 5/20
563/563 [==============================] - 31s 55ms/step - loss: 0.3072 - accuracy: 0.8919 - val_loss: 0.5113 - val_accuracy: 0.8473
Epoch 6/20
563/563 [==============================] - 31s 55ms/step - loss: 0.2866 - accuracy: 0.8986 - val_loss: 0.5102 - val_accuracy: 0.8503
Epoch 7/20
563/563 [==============================] - 31s 55ms/step - loss: 0.2876 - accuracy: 0.8989 - val_loss: 0.5137 - val_accuracy:

In [25]:
model2 = Sequential([
                    Conv1D(32, 3, input_shape=(15, 10), ), 
                    Conv1D(64, 3),
                    Conv1D(128, 3),
                    Conv1D(256, 3),
                    LSTM(500  ),
                    Dense(2000, 'tanh'),
                    Dense(1000, 'tanh'),
                    Dense(35, 'softmax' )
])

In [26]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 13, 32)            992       
                                                                 
 conv1d_4 (Conv1D)           (None, 11, 64)            6208      
                                                                 
 conv1d_5 (Conv1D)           (None, 9, 128)            24704     
                                                                 
 conv1d_6 (Conv1D)           (None, 7, 256)            98560     
                                                                 
 lstm_1 (LSTM)               (None, 500)               1514000   
                                                                 
 dense_3 (Dense)             (None, 2000)              1002000   
                                                                 
 dense_4 (Dense)             (None, 1000)             

In [27]:
model2.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

In [28]:
mchp2 = ModelCheckpoint('drive/MyDrive/datasets/casas-dataset-extracted/csh117/model2.h5', save_best_only=True)

In [29]:
history = model2.fit(X_train, y_train, epochs=50, batch_size=500, validation_data=(X_test, y_test), callbacks=[mchp2])

Epoch 1/50
563/563 [==============================] - 15s 22ms/step - loss: 1.2086 - accuracy: 0.6098 - val_loss: 1.1341 - val_accuracy: 0.6287
Epoch 2/50
563/563 [==============================] - 12s 21ms/step - loss: 1.0452 - accuracy: 0.6527 - val_loss: 1.0769 - val_accuracy: 0.6345
Epoch 3/50
563/563 [==============================] - 11s 20ms/step - loss: 0.9885 - accuracy: 0.6670 - val_loss: 0.9941 - val_accuracy: 0.6602
Epoch 4/50
563/563 [==============================] - 11s 20ms/step - loss: 0.9384 - accuracy: 0.6801 - val_loss: 0.9374 - val_accuracy: 0.6788
Epoch 5/50
563/563 [==============================] - 11s 20ms/step - loss: 0.8919 - accuracy: 0.6936 - val_loss: 0.9305 - val_accuracy: 0.6796
Epoch 6/50
563/563 [==============================] - 12s 20ms/step - loss: 0.8413 - accuracy: 0.7093 - val_loss: 0.8560 - val_accuracy: 0.7053
Epoch 7/50
563/563 [==============================] - 12s 21ms/step - loss: 0.7881 - accuracy: 0.7250 - val_loss: 0.8228 - val_accuracy: